In [14]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
train_path='/content/drive/My Drive/real_and_fake_face'
pred_path='/content/drive/My Drive/test_real_fake'

In [25]:
classes = ['FAKE_FAKE', 'REAL_REAL']

In [26]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.keep = 0.5

        self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=8)
        self.relu1=nn.ReLU()
        self.pool1=nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(num_features=16)
        self.relu2=nn.ReLU()

        self.conv3=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()

        self.pool2=nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.conv4=nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3,stride=1,padding=1)
        self.bn4=nn.BatchNorm2d(num_features=16)
        self.relu4=nn.ReLU()
        

        self.flatten = nn.Flatten()

        self.fc=nn.Linear(in_features=33*33*16,out_features=625, bias=True)
        self.relu5=nn.ReLU()
        self.dout=nn.Dropout(p=1 - self.keep)

        self.outputs = torch.nn.Linear(625, 2, bias=True)
        
                
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool1(output)
            
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)

        output=self.pool2(output)

        output=self.conv4(output)
        output=self.bn4(output)
        output=self.relu4(output)

        output=self.flatten(output)
                        
        #output=output.view(-1,32*75*75)
        output=self.fc(output)


        output=self.relu5(output)
        output=self.dout(output)
            
            
        #output=self.fc(output)
        output=self.outputs(output) 

        return output

In [27]:
model = network()
model=torch.load('/content/drive/MyDrive/Models/my_best_model.model', map_location="cpu")
# model=network(num_classes=2)
# model.load_state_dict(checkpoint)
model.eval()

network(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=17424, out_fea

In [28]:
transformer=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], 
                        [0.5,0.5,0.5])
])

In [29]:
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [30]:
images_path=glob.glob(pred_path+'/*.jpg')

In [31]:
images_path

['/content/drive/My Drive/test_real_fake/easy_101_0010.jpg',
 '/content/drive/My Drive/test_real_fake/easy_103_1111.jpg',
 '/content/drive/My Drive/test_real_fake/easy_102_0101.jpg',
 '/content/drive/My Drive/test_real_fake/easy_100_1111.jpg',
 '/content/drive/My Drive/test_real_fake/easy_106_0011.jpg',
 '/content/drive/My Drive/test_real_fake/easy_10_0001.jpg',
 '/content/drive/My Drive/test_real_fake/easy_105_1100.jpg',
 '/content/drive/My Drive/test_real_fake/easy_107_1011.jpg',
 '/content/drive/My Drive/test_real_fake/easy_108_1111.jpg',
 '/content/drive/My Drive/test_real_fake/easy_104_1000.jpg',
 '/content/drive/My Drive/test_real_fake/hard_10_1111.jpg',
 '/content/drive/My Drive/test_real_fake/hard_1_1111.jpg',
 '/content/drive/My Drive/test_real_fake/hard_7_1111.jpg',
 '/content/drive/My Drive/test_real_fake/hard_8_1000.jpg',
 '/content/drive/My Drive/test_real_fake/hard_6_1110.jpg',
 '/content/drive/My Drive/test_real_fake/hard_2_1111.jpg',
 '/content/drive/My Drive/test_real_

In [32]:
pred_dict={}

for i in images_path:
    pred_dict[i[39:]]=prediction(i,transformer)

In [33]:
pred_dict

{'easy_100_1111.jpg': 'FAKE_FAKE',
 'easy_101_0010.jpg': 'FAKE_FAKE',
 'easy_102_0101.jpg': 'FAKE_FAKE',
 'easy_103_1111.jpg': 'FAKE_FAKE',
 'easy_104_1000.jpg': 'FAKE_FAKE',
 'easy_105_1100.jpg': 'FAKE_FAKE',
 'easy_106_0011.jpg': 'FAKE_FAKE',
 'easy_107_1011.jpg': 'FAKE_FAKE',
 'easy_108_1111.jpg': 'FAKE_FAKE',
 'easy_10_0001.jpg': 'FAKE_FAKE',
 'hard_10_1111.jpg': 'FAKE_FAKE',
 'hard_1_1111.jpg': 'FAKE_FAKE',
 'hard_2_1111.jpg': 'FAKE_FAKE',
 'hard_3_1111.jpg': 'FAKE_FAKE',
 'hard_4_1111.jpg': 'FAKE_FAKE',
 'hard_5_1111.jpg': 'FAKE_FAKE',
 'hard_6_1110.jpg': 'FAKE_FAKE',
 'hard_7_1111.jpg': 'FAKE_FAKE',
 'hard_8_1000.jpg': 'FAKE_FAKE',
 'hard_9_1111.jpg': 'FAKE_FAKE',
 'mid_10_1111.jpg': 'FAKE_FAKE',
 'mid_1_1101.jpg': 'FAKE_FAKE',
 'mid_2_1101.jpg': 'FAKE_FAKE',
 'mid_3_1100.jpg': 'FAKE_FAKE',
 'mid_4_1111.jpg': 'FAKE_FAKE',
 'mid_5_0011.jpg': 'FAKE_FAKE',
 'mid_6_1111.jpg': 'FAKE_FAKE',
 'mid_7_1100.jpg': 'FAKE_FAKE',
 'mid_8_1111.jpg': 'FAKE_FAKE',
 'mid_9_1110.jpg': 'FAKE_FAKE',